In [1]:
import cv2
import os
from os.path import join, isfile
from os import makedirs
from pathlib import Path
import xml.etree.ElementTree as ET
from copy import deepcopy
from tqdm.notebook import tqdm_notebook as ntqdm
import random
from natsort import natsorted
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

plt.rcParams['figure.figsize'] = [15, 10]


In [2]:
data_path = '/home/masoud/Desktop/C'
video_path = Path(data_path) / 'train'
annotation_path = Path(data_path) / 'game-status'

annotations = natsorted(list(annotation_path.glob('*.xml')), key=lambda x: x.stem)
videos = natsorted(list(video_path.glob('*')), key=lambda x: x.stem)
videos = [vid for vid in videos if vid.stem in [annot.stem for annot in annotations]]

pairs = list(zip(videos, annotations))


In [70]:
get_label = lambda annots, label_name: [f.get('name') for f in annots if f.find('tag').get('label') == label_name]

def get_frame_nos(annot_path, label='serving-start'):
    # Parse file
    data = ET.parse(annot_path)
    # Separate all images tags
    images = data.findall('image')
    # Filter the images with no image_tag
    images = [img for img in images if img.find('tag') is not None]
    # Get the image frame number of the ones that matches the label given like `serving-start`, ....
    frame_str = get_label(images, label)
    frame_nos = [int(f.split('_')[1]) for f in frame_str]
    return frame_nos

def random_start_frame(st, end, divisions=5, vdo_length=50):
    arr = np.arange(st, end - vdo_length, dtype=np.int32)
    arrays = np.array_split(arr, divisions)
    # print(st, end)
    # for r in arrays:
    #     print(r)
    results = [random.choice(r.tolist()) for r in arrays]
    return results


# Finding the annotation mistakes

In [6]:
# annot = annotations[7]

# serves = get_frame_nos(annot, 'serving-start')
# end_serves = get_frame_nos(annot, 'serving-end')
# end_games = get_frame_nos(annot, 'in-game-end')

# max_length = max(len(serves), len(end_serves), len(end_games))

# print(annot)
# print("serves: ", len(serves))
# print("end-serves: ", len(end_serves))
# print("end-games: ", len(end_games))


# all_periods = []

# print("prev_end_game | serve-start | end-serve | end_game |")
# for i in range(max_length):
#     eg = end_games[i]
    
#     next_s = serves[i+1]
#     next_es = end_serves[i+1]
#     next_eg = end_games[i+1]
    
#     print(f"{eg}           |     {next_s}     |   {next_es}     |   {next_eg}    |")
#     assert eg < next_s, f"the end_game {eg} - start_serve {next_s}"
#     assert next_s < next_es, f"The end_serve {next_es} - start serve {next_s}"
#     assert next_es < next_eg, f"the end_serve {next_es} is after end_game {next_eg}"
    

/home/masoud/Desktop/C/game-status/13.xml
serves:  226
end-serves:  226
end-games:  226
prev_end_game | serve-start | end-serve | end_game |
237           |     368     |   411     |   510    |
510           |     653     |   716     |   899    |
899           |     980     |   1031     |   1144    |
1144           |     1252     |   1320     |   1438    |
1438           |     1529     |   1618     |   1753    |
1753           |     1857     |   1928     |   2091    |
2091           |     2147     |   2214     |   2308    |
2308           |     2486     |   2523     |   2642    |
2642           |     2783     |   2832     |   2998    |
2998           |     3057     |   3132     |   3271    |
3271           |     3390     |   3432     |   3549    |
3549           |     3658     |   3716     |   3899    |
3899           |     3942     |   4036     |   4187    |
4187           |     4268     |   4336     |   4448    |
4448           |     4572     |   4620     |   4639    |
4639          

IndexError: list index out of range

# Generate Each Label videos (services/in-play/no-play)

In [72]:
output_base_dir = '/home/masoud/Desktop/projects/volleyball_analytics/data/raw/game-status'
service_dir = Path(output_base_dir) / 'service'
game_dir = Path(output_base_dir) / 'play'
no_game_dir = Path(output_base_dir) / 'no-play'
vdo_length = 50

def write_video(cap: cv2.VideoCapture, st: int, end: int, fps: int, w: int, h: int, video_length: int, output_path: Path, video: Path):
    cap.set(1, st)
    filename = f'inplay_{video.stem}_st1_{st}_end_{st+50}.mp4'
    output_file = (output_path / filename).as_posix()
    writer = cv2.VideoWriter(
        output_file,
        cv2.VideoWriter_fourcc(*'mp4v'),
        fps,
        (w, h)
    )
    for i in range(video_length):
        status, frame = cap.read()
        if status:
            writer.write(frame)

    writer.release()

for video, annot in pairs:
    cap = cv2.VideoCapture(video.as_posix())
    w, h, fps = [int(cap.get(i)) for i in range(3, 6)]
    
    serves = get_frame_nos(annot, 'serving-start')
    end_serves = get_frame_nos(annot, 'serving-end')
    end_games = get_frame_nos(annot, 'in-game-end')
    data = {
    'serve': serves, 'end_serve': end_serves, 'end_game': end_games
    }
    df = pd.DataFrame(data=data)

    df['next_serve'] = None
    
    for i in range(len(df)):
        if i != len(df) - 1:
            next_serve = df.at[i+1, 'serve']
            df.at[i, 'next_serve'] = next_serve
        else:
            df.at[i, 'next_serve'] = None
    
    pbar = ntqdm(total=len(df))
    pbar.set_description(f"Processing {video.name}")
    for i, row in df.iterrows():
        pbar.update(1)
        start_serve_fno = row['serve']
        end_serve_fno = start_inplay_fno = row['end_serve']
        end_inplay_fno = start_noplay_fno = row['end_game']
        end_noplay_fno = row['next_serve']
        
        ######################## Generate service videos #################################
        pbar.set_description(f"1/3: {video.name} generating services")
        length = end_serve_fno - start_serve_fno
        cap.set(1, start_serve_fno)

        filename = f'serve_{video.stem}_st_{start_serve_fno}_end_{end_serve_fno}.mp4'
        output_file = (service_dir / filename).as_posix()
        writer = cv2.VideoWriter(
            output_file, 
            cv2.VideoWriter_fourcc(*'mp4v'),
            fps,
            (w, h)
        )
        for i in range(length):
            status, frame = cap.read()
            if status:
                writer.write(frame)

        writer.release()

        ######################## Generate in-play videos #################################
        pbar.set_description(f"2/3: {video.name} generating in-play videos")
        length = end_inplay_fno - start_inplay_fno 
        # print(length)
        if length < 52:
            continue
        elif 52 <= length <= 120:
            # Generate 1 video.
            st = random_start_frame(start_inplay_fno, end_inplay_fno, divisions=1, vdo_length=vdo_length)
            write_video(
                cap=cap,
                st=st[0],
                end=st[0] + 50,
                fps=fps,
                w=w,
                h=h,
                video_length=vdo_length,
                output_path=game_dir,
                video=video
            )

        elif 120 < length <= 500:
            start_frames = random_start_frame(start_inplay_fno, end_inplay_fno, divisions=2, vdo_length=vdo_length)
            for f in start_frames:
                write_video(
                    cap=cap, 
                    st=f, 
                    end=f + 50, 
                    fps=fps, 
                    w=w, 
                    h=h, 
                    video_length=vdo_length, 
                    output_path=game_dir, 
                    video=video
                )

        elif length > 500:
            st = start_inplay_fno
            end = end_inplay_fno
            
            start_frames = random_start_frame(st, end, divisions=5, vdo_length=vdo_length)
            for f in start_frames:
                write_video(
                    cap=cap,
                    st=f,
                    end=f + 50,
                    fps=fps,
                    w=w,
                    h=h,
                    video_length=vdo_length,
                    output_path=game_dir, 
                    video=video
                )

        ######################### Generate no-play videos #################################
        pbar.set_description(f"3/3: {video.name} generating no-play videos")

        if end_noplay_fno is None or start_noplay_fno is None:
            continue
        length = end_noplay_fno - start_noplay_fno
        init_st = start_noplay_fno
        end = end_noplay_fno

        if length < 52:
            continue
        elif 52 <= length <= 120:
            # Generate 1 video.
            st = random_start_frame(init_st, end, divisions=1, vdo_length=vdo_length)
            write_video(
                cap=cap,
                st=st[0],
                end=st[0] + 50,
                fps=fps,
                w=w,
                h=h,
                video_length=vdo_length,
                output_path=no_game_dir,
                video=video
            )

        elif 120 < length <= 500:
            start_frames = random_start_frame(init_st, end, divisions=2, vdo_length=vdo_length)
            for f in start_frames:
                write_video(
                    cap=cap, 
                    st=f, 
                    end=f + 50, 
                    fps=fps, 
                    w=w, 
                    h=h, 
                    video_length=vdo_length, 
                    output_path=no_game_dir, 
                    video=video
                )

        elif length > 500:
            start_frames = random_start_frame(init_st, end, divisions=5, vdo_length=vdo_length)
            for f in start_frames:
                write_video(
                    cap=cap,
                    st=f,
                    end=f + 50,
                    fps=fps,
                    w=w,
                    h=h,
                    video_length=vdo_length,
                    output_path=no_game_dir, 
                    video=video
                )
    pbar.close()

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/42 [00:00<?, ?it/s]

  0%|          | 0/42 [00:00<?, ?it/s]

  0%|          | 0/134 [00:00<?, ?it/s]

  0%|          | 0/179 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/226 [00:00<?, ?it/s]